<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Gmail - Create draft email

**Tags:** #gmail #email #draft #create #python #library

**Author:** [Sriniketh Jayasendil](https://www.linkedin.com/in/sriniketh-jayasendil/)

**Description:** This notebook will show how to create a draft email using the Gmail API. It is usefull for organizations that need to automate the creation of emails.

**References:**
- [Gmail API Documentation](https://developers.google.com/gmail/api/quickstart/python)
- [Gmail API Python Quickstart](https://developers.google.com/gmail/api/quickstart/python)

## Input

### Import libraries

In [ ]:
import naas
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import datetime
import os.path
import base64
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

### Setup Variables
- `SCOPES`: list of scopes to be used for the Gmail API
- `credentials`: credentials to be used for the Gmail API

For more information on how to setup the credentials, please refer to the [Gmail API Python Quickstart](https://developers.google.com/gmail/api/quickstart/python).

In [ ]:
SCOPES = ['https://mail.google.com/']
CLIENT_SECRET_FILE = "./secrets-gmail.json"
email_from = naas.secret.get("GMAIL_FROM_EMAIL")
email_to = naas.secret.get("GMAIL_TO_EMAIL")
email_cc = naas.secret.get("GMAIL_CC_EMAIL")
email_subject = 'Hello Gmail API'
msg_body = 'Hello!! You work super cool!!'

## Model

### Connect to service

In [ ]:
API_NAME ='gmail'
API_VERSION = 'v1'

# Create service account
def Create_Service(client_secret_file, api_name, api_version, *scopes):
    print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)

    cred = None

    pickle_file = f'token_{API_SERVICE_NAME}_{API_VERSION}.pickle'

    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()

        with open(pickle_file, 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME, 'service created successfully')
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None
    
service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

## Output

### Create draft email

This function will create a draft email using the Gmail API.

In [ ]:
mimeMessage = MIMEMultipart()
mimeMessage['from'] = email_from
mimeMessage['to'] = email_to
mimeMessage['cc'] = email_cc
mimeMessage['subject'] = email_subject
mimeMessage.attach(MIMEText(msg_body, 'plain'))
raw_string = base64.urlsafe_b64encode(mimeMessage.as_bytes()).decode()
response = service.users().drafts().create(
    userId='me',
    body={'message': {'raw': raw_string }}
).execute()

# Display result
print(response)